In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
bank_accounts = pd.read_csv('data/shopee-fraud-detection-bank-accounts.csv', low_memory=False)
credit_cards = pd.read_csv('data/shopee-fraud-detection-credit-cards.csv')
devices = pd.read_csv('data/shopee-fraud-detection-devices.csv')
orders = pd.read_csv('data/shopee-fraud-detection-orders.csv')

In [3]:
bank_accounts

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


In [4]:
credit_cards

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018
...,...,...
38703,25879226,839199xxxxxx3838|12-2022
38704,9331492,398138xxxxxx2208|08-2019
38705,2890486,839330xxxxxx9180|08-2022
38706,107429252,899933xxxxxx2930|06-2021


In [5]:
devices

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=
...,...,...
1430306,7061456,ZSTXD6wfHN/1qPVqMrIbwHYX5ELWlUPO3NFXCHK50Mc=
1430307,26416970,o+9gbX201ITS3MuMZYjuZUzV7sOoR57/0ttyX3eSWTA=
1430308,79633116,NZhNiXYhArpPquzrqh/kVj2IZ3QPRKm6FH90vsTlhiy=
1430309,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=


In [6]:
orders

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


In [7]:
from itertools import permutations

In [8]:
def FindSharedData(df):
    col = df.columns[1]
    x = df.groupby(col)['userid'].nunique()
    x = x[x > 1]
    x = df[df.iloc[:, 1].isin(x.index)]
    
    y = pd.DataFrame([
        [a, n, b]
        for n, g in x.groupby(col)
        for a, b in permutations(g['userid'], 2)
    ], columns=['userid1', col, 'userid2'])

    y = y.astype({col:'str'})
    y.sort_values(col, inplace=True)
    
    return y

shared_credit_cards = FindSharedData(credit_cards)
shared_bank_accounts = FindSharedData(bank_accounts)
shared_devices = FindSharedData(devices)

In [9]:
shared_credit_cards

,userid1,credit_card,userid2
0,24704966,301348xxxxxx1098|01-2020,14202676
1,24704966,301348xxxxxx1098|01-2020,46012496
2,14202676,301348xxxxxx1098|01-2020,24704966
3,14202676,301348xxxxxx1098|01-2020,46012496
4,46012496,301348xxxxxx1098|01-2020,24704966
...,...,...,...
2889,25359646,899949xxxxxx0092|07-2020,25361240
2890,36010164,899949xxxxxx2303|05-2019,34757966
2891,34757966,899949xxxxxx2303|05-2019,36010164
2892,18511368,899949xxxxxx8223|09-2019,25511320


In [10]:
shared_bank_accounts

,userid1,bank_account,userid2
0,54966514,0000000000,21407120
1,54966514,0000000000,19901940
2,21407120,0000000000,54966514
3,21407120,0000000000,19901940
4,19901940,0000000000,54966514
...,...,...,...
65217,130715038,999802000402802,99930758
65218,43665900,9999028800,143737034
65219,143737034,9999028800,43665900
65220,44047452,999993229,13167570


In [11]:
shared_devices

,userid1,device,userid2
0,102164030,+++ivFTF+M/DxnA21MRSxuqZO/KUheIu0RXva/O41sq=,66422306
1,66422306,+++ivFTF+M/DxnA21MRSxuqZO/KUheIu0RXva/O41sq=,102164030
2,11734268,++53iBcrIz8AlMWkrmV+wM2Dd6sDURson4MMwahyFRW=,15265100
3,11734268,++53iBcrIz8AlMWkrmV+wM2Dd6sDURson4MMwahyFRW=,10029066
4,15265100,++53iBcrIz8AlMWkrmV+wM2Dd6sDURson4MMwahyFRW=,11734268
...,...,...,...
185093,142715060,zzr+Gj7BPyvux9jp1XDLpxUY2qUz9APDuNwYdwKKwsq=,190351570
185094,106469010,zzruABZMnkkG+PacllaAU8jLxR4IYYJS+TIMmIfA85A=,94431850
185095,94431850,zzruABZMnkkG+PacllaAU8jLxR4IYYJS+TIMmIfA85A=,106469010
185096,9878070,zzzLGRdBMVYjnK9E7bC3xlOS8nUF7oB8uhKBO33EbQW=,129766014


In [12]:
link = pd.merge(shared_credit_cards, shared_bank_accounts, how='outer', on=['userid1', 'userid2'])
link = pd.merge(link, shared_devices, how='outer', on=['userid1', 'userid2'])

link['link_type'] = np.nan
link.loc[~link['credit_card'].isna() & link['link_type'].isna(), 'link_type'] = 'credit_card'
link.loc[~link['bank_account'].isna() & link['link_type'].isna(), 'link_type'] = 'bank_account'
link.loc[~link['device'].isna() & link['link_type'].isna(), 'link_type'] = 'device'

link.loc[link['credit_card'].isna(), 'credit_card'] = link.loc[link['credit_card'].isna(), 'bank_account']
link.loc[link['credit_card'].isna(), 'credit_card'] = link.loc[link['credit_card'].isna(), 'device']
link.drop(['bank_account', 'device'], axis=1, inplace=True)
link.rename(columns={'credit_card':'link'}, inplace=True)
link = link[['userid1', 'link_type', 'link', 'userid2']]

link.drop_duplicates(['userid1', 'userid2'], inplace=True)
link

,userid1,link_type,link,userid2
0,24704966,credit_card,301348xxxxxx1098|01-2020,14202676
1,24704966,credit_card,301348xxxxxx1098|01-2020,46012496
9,14202676,credit_card,301348xxxxxx1098|01-2020,24704966
10,14202676,credit_card,301348xxxxxx1098|01-2020,46012496
11,46012496,credit_card,301348xxxxxx1098|01-2020,24704966
...,...,...,...,...
246265,82331972,device,zxUvh+euPY5vdutOTfHEhAxp8Vb5GZB1+f2L9RmBb5y=,8802740
246266,33152428,device,zxUvh+euPY5vdutOTfHEhAxp8Vb5GZB1+f2L9RmBb5y=,82331972
246267,8802740,device,zxUvh+euPY5vdutOTfHEhAxp8Vb5GZB1+f2L9RmBb5y=,82331972
246268,34150580,device,zyJR/F5sDPgxL1+gwoRKyJ0mm8b5nDjNpoZgcPO7L9g=,10371690


In [13]:
def to_set(x):
    return set(x)

def merge(lsts):
    sets = lsts.tolist()
    merged = True
    while merged:
        merged = False
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = True
                    common = common | x
            results.append(common)
        sets = results
    sets = pd.Series(sets)
    return sets

In [14]:
group = link.groupby('link')['userid1'].agg(to_set)
groups_of_people = merge(group)
groups_of_people[-10:]   # check

22746               {30308546, 195996292}
22747              {216355698, 194476740}
22748                {82682160, 55603306}
22749                {73515392, 54487694}
22750    {231756696, 23732746, 235380218}
22751               {71552930, 175264522}
22752              {198845510, 138138078}
22753                {30029008, 57648462}
22754              {166868706, 166866630}
22755                {10371690, 34150580}
dtype: object

In [15]:
userid = [person for group in groups_of_people for person in group]
group = np.repeat(groups_of_people.values, groups_of_people.str.len())
groups = pd.DataFrame({'userid':userid, 'group':group})
groups

,userid,group
0,152997888,"{152997888, 171573250, 170278914, 160161796, 5..."
1,171573250,"{152997888, 171573250, 170278914, 160161796, 5..."
2,170278914,"{152997888, 171573250, 170278914, 160161796, 5..."
3,160161796,"{152997888, 171573250, 170278914, 160161796, 5..."
4,54951944,"{152997888, 171573250, 170278914, 160161796, 5..."
...,...,...
58647,57648462,"{30029008, 57648462}"
58648,166868706,"{166868706, 166866630}"
58649,166866630,"{166868706, 166866630}"
58650,10371690,"{10371690, 34150580}"


In [16]:
buyer_groups = pd.merge(orders, groups, how='inner', left_on='buyer_userid', right_on='userid')
buyer_groups = buyer_groups[['orderid', 'buyer_userid', 'seller_userid', 'group']]
seller_groups = pd.merge(orders, groups, how='inner', left_on='seller_userid', right_on='userid')
seller_groups = seller_groups[['orderid', 'group']]

order_groups = pd.merge(buyer_groups, seller_groups, how='inner', on='orderid')
order_groups['linked'] = order_groups['group_x']==order_groups['group_y']
order_groups = order_groups[order_groups['linked']==True]
order_groups

,orderid,buyer_userid,seller_userid,group_x,group_y,linked
297,1954198318,221232712,66353306,"{221232712, 66353306}","{221232712, 66353306}",True
474,1955598428,35545436,70763052,"{70763052, 35545436}","{70763052, 35545436}",True
638,1954515646,32834366,188151804,"{64069730, 30010466, 86817350, 43613612, 30010...","{64069730, 30010466, 86817350, 43613612, 30010...",True
639,1954554226,32834366,188151804,"{64069730, 30010466, 86817350, 43613612, 30010...","{64069730, 30010466, 86817350, 43613612, 30010...",True
1163,1953728724,168491444,158559422,"{168491444, 158559422}","{168491444, 158559422}",True
...,...,...,...,...,...,...
34516,1953753856,77758590,12950834,"{152997888, 171573250, 170278914, 160161796, 5...","{152997888, 171573250, 170278914, 160161796, 5...",True
34527,1953994554,30153396,129295230,"{178244640, 30153396, 129295230}","{178244640, 30153396, 129295230}",True
34579,1955929340,235579716,80629986,"{80629986, 235579716}","{80629986, 235579716}",True
34661,1953812320,235299524,229915488,"{200818688, 164136960, 224722946, 118749188, 1...","{200818688, 164136960, 224722946, 118749188, 1...",True


In [17]:
original_datalink = order_groups[['orderid', 'seller_userid', 'buyer_userid']].copy()
original_datalink.rename(columns={'buyer_userid':'userid1'}, inplace=True)
original_datalink

,orderid,seller_userid,userid1
297,1954198318,66353306,221232712
474,1955598428,70763052,35545436
638,1954515646,188151804,32834366
639,1954554226,188151804,32834366
1163,1953728724,158559422,168491444
...,...,...,...
34516,1953753856,12950834,77758590
34527,1953994554,129295230,30153396
34579,1955929340,80629986,235579716
34661,1953812320,229915488,235299524


In [18]:
link = link.astype('str')
link.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118604 entries, 0 to 246269
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userid1    118604 non-null  object
 1   link_type  118604 non-null  object
 2   link       118604 non-null  object
 3   userid2    118604 non-null  object
dtypes: object(4)
memory usage: 4.5+ MB


In [19]:
original_datalink = original_datalink.astype('str')
original_datalink.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 297 to 34677
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   orderid        478 non-null    object
 1   seller_userid  478 non-null    object
 2   userid1        478 non-null    object
dtypes: object(3)
memory usage: 14.9+ KB


In [20]:
def GetLink(row, supporting_data):
    current_datalink = pd.DataFrame(row).T
    keep_looping = True
    col = []
    i = 1   # for naming columns
    
    while keep_looping:
        x = pd.merge(current_datalink, supporting_data, how='left', on='userid1')
        supporting_data = supporting_data[~supporting_data['userid1'].isin(x['userid1'])].copy()
        x.rename(columns={'userid1':'userid', 
                          'link_type':'link_type'+str(i),
                          'link':'link'+str(i),
                          'userid2':'userid1'}, inplace=True)
        
        final = x[x['seller_userid']==x['userid1']].copy()
        
        if len(final) != 0:
            keep_looping = False
        
        current_datalink = x
        col.append('link_type'+str(i))
        col.append('link'+str(i))
        i = i + 1
        
    final.replace({'credit_card':'a', 'bank_account':'b', 'device':'c'}, inplace=True)
    final.sort_values(col, inplace=True)
    final.replace({'a':'credit_card', 'b':'bank_account', 'c':'device'}, inplace=True)
    final.drop_duplicates('orderid', inplace=True)
    final.columns = np.arange(len(final.columns))
    return final

In [21]:
final_datalink = []
for _, row in tqdm(original_datalink.iterrows(), total=len(original_datalink)):
    final_datalink.append(GetLink(row, link))

100%|████████████████████████████████████████████████████████████████████████████████| 478/478 [01:17<00:00,  6.16it/s]


In [22]:
final = pd.concat(final_datalink)
final

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,1954198318,66353306,221232712,bank_account,8300298809,66353306,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1955598428,70763052,35545436,device,/3TLpeou8xXsNxpACFFKr34Kqqwxiu5Hi1keJ6plk5E=,70763052,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1954515646,188151804,32834366,device,1KNEOFRIZaFcFx5+S+bOxyWuWBbITxnfoM7j6aZg05y=,188151804,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1954554226,188151804,32834366,device,1KNEOFRIZaFcFx5+S+bOxyWuWBbITxnfoM7j6aZg05y=,188151804,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1953728724,158559422,168491444,device,yf7AHm3O97XAQwQuSmyoaxcaFSSAZcVCxm5YNgiEo4S=,158559422,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10048,1953753856,12950834,77758590,device,OInyTV02J4g+mOSuLq8bZqzHMio1mSj7zxGhHrjcAHI=,5260840,device,eVjihNHT0ano6PXp4s9F/ym9cFXQtnoHyDImKSs0kUK=,44563210,device,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1953994554,129295230,30153396,device,U+J33IBG2WOcbS1jGj2GdsLQb7GEilLaKjHIL29U0b0=,129295230,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1955929340,80629986,235579716,device,4frUznPkj1NuGyDLkPfovV3mHm1MSGxIrXaz68I4n4m=,80629986,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1953812320,229915488,235299524,device,QWPloLVNUPpaxlUjlJQytCXEc5FLy7HkzCcCX0DWdzg=,229915488,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
final.fillna('', inplace=True)
final['is_fraud'] = ''
final['is_fraud'] = final.iloc[:, 2]

for j in range(3, final.shape[1]-1, 1):
    condition = final.iloc[:, j] != ''
    
    if j % 3 == 0:
        final.loc[condition, 'is_fraud'] = \
        final.loc[condition, 'is_fraud'].str.cat(final.loc[condition, j], sep = "-\"")
    elif j % 3 == 1:
        final.loc[condition, 'is_fraud'] = \
        final.loc[condition, 'is_fraud'].str.cat(final.loc[condition, j], sep = ":")
    else:
        final.loc[condition, 'is_fraud'] = \
        final.loc[condition, 'is_fraud'].str.cat(final.loc[condition, j], sep = "\"->")

final.drop(final.columns[1:-1], axis=1, inplace=True)

In [24]:
final.columns = ['orderid', 'is_fraud']
final

,orderid,is_fraud
0,1954198318,"221232712-""bank_account:8300298809""->66353306"
0,1955598428,"35545436-""device:/3TLpeou8xXsNxpACFFKr34Kqqwxi..."
1,1954515646,"32834366-""device:1KNEOFRIZaFcFx5+S+bOxyWuWBbIT..."
1,1954554226,"32834366-""device:1KNEOFRIZaFcFx5+S+bOxyWuWBbIT..."
0,1953728724,"168491444-""device:yf7AHm3O97XAQwQuSmyoaxcaFSSA..."
...,...,...
10048,1953753856,"77758590-""device:OInyTV02J4g+mOSuLq8bZqzHMio1m..."
0,1953994554,"30153396-""device:U+J33IBG2WOcbS1jGj2GdsLQb7GEi..."
0,1955929340,"235579716-""device:4frUznPkj1NuGyDLkPfovV3mHm1M..."
0,1953812320,"235299524-""device:QWPloLVNUPpaxlUjlJQytCXEc5FL..."


In [25]:
orders = pd.read_csv('data/shopee-fraud-detection-orders.csv', dtype='str')
orders = pd.merge(orders, final, how='left', on='orderid')
orders.fillna('not fraud', inplace=True)
orders[orders['is_fraud']!='not fraud']

,orderid,buyer_userid,seller_userid,is_fraud
1649,1954198318,221232712,66353306,"221232712-""bank_account:8300298809""->66353306"
2679,1955598428,35545436,70763052,"35545436-""device:/3TLpeou8xXsNxpACFFKr34Kqqwxi..."
3545,1954515646,32834366,188151804,"32834366-""device:1KNEOFRIZaFcFx5+S+bOxyWuWBbIT..."
5938,1953728724,168491444,158559422,"168491444-""device:yf7AHm3O97XAQwQuSmyoaxcaFSSA..."
8393,1955955178,235599454,51098362,"235599454-""bank_account:9120282009""->51098362"
...,...,...,...,...
612805,1955929340,235579716,80629986,"235579716-""device:4frUznPkj1NuGyDLkPfovV3mHm1M..."
614563,1954626246,197784360,12132476,"197784360-""device:79NZWlJcqP6R4VsNMt0ZNnMfURvO..."
616320,1953812320,235299524,229915488,"235299524-""device:QWPloLVNUPpaxlUjlJQytCXEc5FL..."
616725,1954821958,111271962,66519400,"111271962-""bank_account:0323129990""->66519400"


In [26]:
submission = orders.drop(['buyer_userid', 'seller_userid'], axis=1)
submission.to_csv('submission_buyer.csv', index=False)